Load libraries

In [146]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Ochtend

Load en bekijk data

In [147]:
# load the data in
df = pd.read_csv('titanic_data.csv')

Bekijk de datatypes

In [148]:
# view only the head of the data 
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Wat is de minimum, maximum en gemiddelde leeftijd van de passagiers?

In [149]:
print("This is the minimum {}".format(df['Age'].min()))
print("This is the mean {}".format(df['Age'].mean()))
print("This is the maximum {}".format(df['Age'].max()))

This is the minimum 0.42
This is the mean 29.69911764705882
This is the maximum 80.0


Wat is het percentage van mensen die de ramp niet hebben overleefd?

In [150]:
survived = df['Survived']

survivers = (survived == 1).sum()
victims = (survived == 0).sum()

precentVictims = victims / (victims + survivers) * 100

print("The precent of victim is {}".format(precentVictims))

The precent of victim is 61.61616161616161


Wat is de minimum, gemiddelde, en maximum prijs (Fare) in de verschillende klassen (Pclass)?

In [151]:
grouped = df.groupby('Pclass')

stats = grouped['Fare'].agg(['min', 'mean', 'max'])

stats

,min,mean,max
Pclass,,,
1,0.0,84.154687,512.3292
2,0.0,20.662183,73.5000
3,0.0,13.675550,69.5500


Welke data is categorisch?

###### Answer: Name, Sex,  Cabin and Ticket

Verijder de kolommen 'Name', 'Cabin' en 'Ticket'

In [152]:
new_df = df.drop(['Name', 'Cabin', 'Ticket'], axis=1)

new_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


Codeer categorische features met indices (getallen)

In [153]:
# factorize certain columns
df[['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']] = df[['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']].apply(lambda x: pd.factorize(x)[0])

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,0,0,22.0,1,0,0,7.2500,-1,0
1,2,1,1,1,1,38.0,1,0,1,71.2833,0,1
2,3,1,3,2,1,26.0,0,0,2,7.9250,-1,0
3,4,1,1,3,1,35.0,1,0,3,53.1000,1,0
4,5,0,3,4,0,35.0,0,0,4,8.0500,-1,0


Vervang NaN in "Ages". Bepaal zelf of mediaan of gemiddelde hiervoor het best is.

In [154]:
# I choose to replace the NaN in "Ages" with the median
# Because there is some outliers that would affect the results

df['Age'].fillna(df['Age'].median(), inplace=True)

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,0,0,22.0,1,0,0,7.2500,-1,0
1,2,1,1,1,1,38.0,1,0,1,71.2833,0,1
2,3,1,3,2,1,26.0,0,0,2,7.9250,-1,0
3,4,1,1,3,1,35.0,1,0,3,53.1000,1,0
4,5,0,3,4,0,35.0,0,0,4,8.0500,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,886,0,27.0,0,0,677,13.0000,-1,0
887,888,1,1,887,1,19.0,0,0,678,30.0000,145,0
888,889,0,3,888,1,28.0,1,2,614,23.4500,-1,0
889,890,1,1,889,0,26.0,0,0,679,30.0000,146,1


## Middag

Voorspel of passagiers de ramp hebben overleefd of niet. Bereid de data voor om je model te trainen. Wat wordt je target vector en hoe ziet je feature matrix er uit?

In [155]:
# my target verctor will be the Column 'Survived'
# my feature matrix would have the other important columns like 
# 'Pclass', 'Age', 'Fare' ...

# features
x = df.drop(['Survived'], axis=1)
x.shape

(891, 11)

In [156]:
# target
y = df['Survived']
y.shape

(891,)

In [157]:
# 20% of the data will be used for testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 6)

Train en test KNN classifier

In [158]:
# create a KNN classifier with k=5
knn = KNeighborsClassifier(n_neighbors = 5)

# fit the classifier to the training data
knn.fit(x_train, y_train)

# make predictions on the test data
y_pred = knn.predict(x_test)

# calculate accuracy of the classifier
accuracy_score(y_pred, y_test)

0.6256983240223464

Train en test regressie classifier. Kies je lineair of logistisch?

In [164]:
# here I'll choose to use logistic regression because we are predicting 
# the chance of survivors so it is not a continuous result

# create the model with the solver 'newton-cg' because i got a warning message
logistic_regression = LogisticRegression(solver = 'newton-cg')

# train the model
logistic_regression.fit(x_train, y_train)

# make predictions on the test data
y_pred = logistic_regression.predict(x_test)

# calculate accuracy of the classifier
accuracy_score(y_pred, y_test)

0.8547486033519553

Welk model geeft het beste resultaat?

###### answer: model logistic regression

Extra: Kan je het resultaat verbeteren?